# Film production.

# 1.0 Introduction

## 1.0.1 Business Understanding

One of the most well-known forms of art is film production, which has millions of spectators worldwide. There are many ways that movies can be communicated to viewers as a result of the development of various films genres over time.

I intend to analyse on the categories of movies that are doing the best at the box office and to translate findings into useful advice for my client and the factors to consider for a movie producer to ensure maximum return on investment.

## 1.0.2 Data Understanding

The dataset for this project was compiled from several sources.The data have been collected over the past years and its from box office and the IMDB database.
These are the sites with accurate movie data and are frequently updated hence the choice for these sites.
These are the datasets I will use for my analysis. 
* Box Office Mojo data
* IMDB  data  
* popular movies data
* top rated movies data

The finals datasets I used for my analysis had 1517 rows, and 9 columns.These were the features for my analysis:
 * release_date 
 * original_title'
 * production_budget
 * domestic_gross
 * worldwide_gross
 * averagerating
 * genres
 * numvotes
 * runtime_minutes
 * studio

Release_date I used to check on how the genres have been changing over the years as well as how the profits have been chaging over time.

Original title was the unique identifier for all the movies in my dataset.I used this to merge most of the datasets and to check for duplicates.

Production budget was crucial for my analysis,I used it for cmparison of how budgets for genres differed at to clasify the top 100 most expensive movies, and to find the correlation between domestic gross.

To analyse profit making movies I used the domestic gross to classify movies as either profit making or loss.

The average rating was to classify movies based on their popularity and how they are rated.

Genres column was to check how gross changes per movie genre

Runtime in minutes was to analyse how movie duration has been changing over time and classify movies.

studio feature was to compare and how it relates to the the worlwide gross.

# 2.0 Data Preparation Process

The data has to be prepared before the anlysys begins.

## 2.0.1 Loading Libraries
All important libraries that I used in my anlysis and connecting to the sqllite database to access the im dataset 

> data sets merging

In [ ]:
# importing libraries
import pandas as pd
import numpy as np

import sqlite3
from dateutil import parser
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from wordcloud import WordCloud
import matplotlib.cm as cm

## 2.0.2 Loading datasets

In [ ]:
# Loading of the data sets.
  
#box office gross dataset
box_office_df = pd.read_csv('Data/bom.movie_gross.csv.gz')

#Teddy Newton movie budgets dataset
movie_budgets_df = pd.read_csv('Data/tn.movie_budgets.csv.gz')

#the movie data base dataset
tmdb_df = pd.read_csv('Data/tmdb.movies.csv.gz')

#Rotten Tomatoes reviews dataset
rt_reviews_df = pd.read_csv('Data/rt.reviews.tsv.gz', sep='\t', encoding = 'unicode_escape')

#Rotten Tomatoes movie info dataset
rt_movie_info_df = pd.read_csv('Data/rt.movie_info.tsv.gz', sep='\t', encoding = 'unicode_escape')


 ##### 2.0.3 SQL

In [ ]:
# connect to database
conn = sqlite3.Connection("Data/im.db")

In [ ]:
# read sqlite query results into a pandas DataFrame
# joining the two tables using the movie_id key

joint_query = """
SELECT *
FROM movie_basics
JOIN movie_ratings USING (movie_id)
;
"""
sql_data = pd.read_sql(joint_query, conn)
DataOne = sql_data.copy(deep=True)

#### 2.0.3.1 dataOne preview

>##### dataOne

In [ ]:
# a function to overview the data

def shape_info(my_df):
    print(f"data frame shape and information:\nrows:    {my_df.shape[0]}\ncolumns:    {my_df.shape[1]}")
    print()
    return my_df.info()

In [ ]:
# dataset information
shape_info(DataOne)

In [ ]:
# function columns datatypes

def columns_dtypes(my_df):
    print(f"Numerical columns:{len(list(my_df.select_dtypes(include= np.number)))}")
    print(f"Categorical columns:{len(list(my_df.select_dtypes(include= 'object')))}")

In [ ]:
# check datatypes
columns_dtypes(DataOne)

#### 2.0.3.2 dataOne cleaning

In [ ]:
# # check all columns distribution plot

# def dist_check(my_data):
#     plt.figure(figsize=(20,22))
#     for i in range(1,13):
#         plt.subplot(5, 4, i)
#         sns.histplot(my_data[my_data.columns[i]], kde=True)
        
# #check distribution data
# dist_check(DataOne)  

In [ ]:
### how are our features distrubuted
def dist_check(my_data):
    num_cols = my_data.select_dtypes(include=["float64", "int64"]).columns
    ncols = len(num_cols)
    plt.figure(figsize=(20, int(22/4 * (ncols/4 + 1))))
    for i, col in enumerate(num_cols):
        plt.subplot(ncols//4 + 1, 4, i + 1)
        sns.histplot(my_data[col], kde=True)
        plt.title(col)
dist_check(DataOne)

In [ ]:
def validate_data(my_data):
    percent_missing = round(my_data.isnull().sum() * 100 / len(my_data),2)
    missing_value_df = pd.DataFrame(percent_missing,columns=['perc_missing'])
    duplicate_rows = my_data.duplicated().sum()
    print(f"duplicates: {duplicate_rows}")
    print('    *** *** ***    ')
    print(f"null values percentage per column:\n{missing_value_df}")
    most_missing_col = missing_value_df[missing_value_df['perc_missing'] == missing_value_df['perc_missing'].max()].index[0]
    print(f"Most missing values column: {most_missing_col}")
    return most_missing_col, missing_value_df, duplicate_rows

# Example usage
most_missing_col, missing_value_df, duplicate_rows = validate_data(DataOne)

In [ ]:
# check duplicates and null

def check_duplicates_missing(dataframe):
    # calculate percentage of missing values
    percent_missing = dataframe.isnull().mean().round(4) * 100
    #calculate percentage of duplicate rows
    percent_duplicates = dataframe.duplicated().mean() * 100
    # create result dataframe
    result = pd.DataFrame({'Missing Values %': percent_missing, 'Duplicate Values %': percent_duplicates})
    # find column with most missing values
    if percent_missing.max() !=0:
        column_most_missing = percent_missing.idxmax()
        print(f"{(column_most_missing).capitalize()} is the column with most missing values:")
        print()
    else:
        print("No column with missing values")
    if percent_duplicates.max() !=0:
        column_most_duplicates = percent_duplicates.idxmax()
        print("Column with most duplicates:",column_most_duplicates)
    else:
        print("No duplicates")
    return result

In [ ]:
check_duplicates_missing(DataOne)

In [ ]:
# # function to column with most null values form data frame

# def find_null_columns(df):
#     null_cols = df.isnull().sum()
#     most_null_col = null_cols[null_cols == null_cols.max()].index[0]
#     return most_null_col

# # calling the funtion
# print(find_null_columns(DataOne))

# 3.0 Data Cleaning

### functions used to clean


In [ ]:
# detect outliers
def detect_outliers_zscore(data, threshold=3):
    """
    Detect outliers in a pandas DataFrame using Z-score method
    :param data: pandas DataFrame
    :param threshold: Z-score threshold, default is 3
    :return: pandas DataFrame indicating the outliers (True)
    """
    # Select only the numerical columns
    num_cols = data.select_dtypes(include=np.number).columns
    num_data = data[num_cols]
    
    # Calculate Z-scores of each numerical column
    z_scores = np.abs(num_data - num_data.mean()) / num_data.std()
    
    # Create a mask indicating the outliers
    outlier_mask = z_scores > threshold
    
    return outlier_mask

def show_outliers_perc(data):
    """
    Display the percentage of outliers per column in a pandas DataFrame
    :param data: pandas DataFrame
    :return: None
    """
    outlier_mask = detect_outliers_zscore(data)
    outlier_perc = 100 * outlier_mask.mean().round(2)
    print(f'Percentage of outliers per numerical column: \n{outlier_perc}')

In [ ]:
# Show the percentage of outliers per numerical column
show_outliers_perc(DataOne)


In [ ]:
# def visualize_outliers_zscore(df, threshold=3):
#     numerical_cols = df.select_dtypes(include=["float64", "int64"]).columns
#     fig, ax = plt.subplots(figsize=(10, 6))
#     for col in numerical_cols:
#         mean = df[col].mean()
#         std = df[col].std()
#         z_scores = (df[col] - mean) / std
#         outliers = df[np.abs(z_scores) > threshold]

#         sns.boxplot(x=df[col], ax=ax)
#         sns.stripplot(x=outliers[col], color='red', ax=ax)
#     plt.rcParams["axes.grid"] = True
#     ax.set_title("Outliers")
#     ax.set_xlabel("Features")
#     plt.show()

# visualize_outliers_zscore(DataOne)

In [ ]:
# # visualize outliers
# import seaborn as sns

# def visualize_outliers_boxplot(data):    
#     # Select only the numerical columns
#     num_cols = data.select_dtypes(include=np.number).columns
#     num_data = data[num_cols]
    
#     # Set the style of the plots to whitegrid
#     sns.set_style('whitegrid')

#     # Plot box plots for each numerical column
#     plt.rcParams["axes.grid"] = True
#     plt.figure(figsize=(15, 10))
#     for i, col in enumerate(num_cols):
#         plt.subplot(3, 3, i+1)
#         sns.boxplot(data=num_data, x=num_data.index, y=col, fliersize=2.5)
#         plt.title(col)
#     plt.tight_layout()
#     plt.show()


In [ ]:
# # visualize outliers per column
# def visualize_outliers_zscore(df, threshold=3):
#     numerical_cols = df.select_dtypes(include=["float64", "int64"]).columns
#     plt.rcParams["axes.grid"] = True
#     for col in numerical_cols:
#         mean = df[col].mean()
#         std = df[col].std()
#         z_scores = (df[col] - mean) / std
#         outliers = df[np.abs(z_scores) > threshold]

#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.boxplot(x=df[col], ax=ax)
#         sns.stripplot(x=outliers[col], color='red', ax=ax)
#         ax.set_title(f"Outliers in {col.title()}")
#         ax.set_xlabel(col)
#         plt.show()


In [ ]:
def visualize_outliers_zscore(df, threshold=3):
    plt.rcParams["axes.grid"] = True
    numerical_cols = df.select_dtypes(include=["float64", "int64"]).columns
    for col in numerical_cols:
        mean = df[col].mean()
        std = df[col].std()
        z_scores = (df[col] - mean) / std
        outliers = df[np.abs(z_scores) > threshold]

        fig, ax = plt.subplots(figsize=(10, 6))
        sns.boxenplot(df[col], ax=ax)
        ax.scatter(x=outliers.index, y=outliers[col], color='red', s=50)
        ax.set_title(f"Outliers in {col.title()}")
        ax.set_xlabel(col)
        plt.show()

visualize_outliers_zscore(DataOne)       

In [ ]:
def corre_plot(df, col=None):
    plt.rcParams["axes.grid"] = True
    corr = df.corr()
    if col:
        corr = corr[col].drop(col)
        corr = corr[abs(corr) > 0.5].sort_values(ascending=False)
    mask = np.triu(np.ones_like(corr, dtype=bool))
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, mask=mask, annot=True, cmap='coolwarm',linewidths=0.5)
    plt.title("Correlation", fontsize=18)
    # show plot
    plt.show()



In [ ]:
corre_plot(DataOne)

In [ ]:
# correlation ascending  oeder
def visualize_correlation(df, column):
    plt.style.use([])
    plt.figure(figsize=(10, 10))
    heatmap = sns.heatmap(df.corr()[[column]].sort_values(by=column, ascending=False),
                          vmin=-1, vmax=1, annot=True, cmap=None)
    heatmap.set_title(f'Features Correlating with {column}', fontdict={'fontsize':18}, pad=16)
    plt.show()


### 3.0.1 combining of the data sets to a final data set

In [ ]:
#the data sets were all combied on common keys before cleaning of the data

joint_query.dropna(inplace = True)

box_office_df.dropna(inplace = True)

box_office_df.rename(columns = {'title':'original_title'}, inplace = True)

combined_data_1 = pd.merge(joint_query, box_office_df, how='inner', on='original_title')

movie_budgets_df.dropna(inplace = True)

movie_budgets_df.rename(columns = {'movie':'original_title'}, inplace = True)
 
# Find the columns that aren't in the first DataFrame
different_cols = combined_data_1.columns.difference(movie_budgets_df.columns)
 
# Filter out the columns that are different.
data3 = combined_data_1[different_cols]
 
# Merge the DataFrames
data = pd.merge(movie_budgets_df, data3, left_index=True,
                     right_index=True, how='inner')
data.shape

In [ ]:
# dropping of data features that I did not want to analyse.
final_data = data.drop(['id','foreign_gross','movie_id','primary_title','start_year','year'], axis=1)

### 3.1 cleaning of the final dataset

#### 3.1.0 fixing structural issues
removal of the **$** from production_budget, domestic_gross and worldwide_gross


In [ ]:
final_data['production_budget'] = final_data['production_budget'].str.replace('[\$\,]','',regex=True)
final_data['domestic_gross'] = final_data['domestic_gross'].str.replace('[\$\,]','',regex=True)
final_data['worldwide_gross'] = final_data['worldwide_gross'].str.replace('[\$\,]','',regex=True)


#### 3.1.1 changing data types 
production_budget, domestic_gross and worldwide_gross

In [ ]:
final_data['production_budget'] = pd.to_numeric(final_data['production_budget']) 
final_data['domestic_gross'] = pd.to_numeric(final_data['domestic_gross'])
final_data['worldwide_gross'] = pd.to_numeric(final_data['worldwide_gross'])

final_data.dtypes

In [ ]:
list(final_data.columns)

### data after cleaning

In [ ]:
final_data.shape

In [ ]:
final_data.info()

## 4.0 Exploratory Data Analysis

### 4.0.1 Profit Calculation

I now want to discuss the relationship between a movie's budget and how much it earns at the box office. I'll focus on the **Domestic gross**, which includes all ticket sales revenue from US and Canadian theaters.

I'll use the Pearson correlation coefficient to measure the linear correlation   between the production  budget and box office domestic gross.


This result indicates a correlation of **0.6** between production budget and the domestic gross.
This implies a **moderate** strength of relationship.

In [ ]:
final_data['domestic_profit'] = final_data['domestic_gross'] - final_data['production_budget']
correlation_df = final_data.iloc[:, [6,2,3,4,10]].head(13)
correlation = final_data.corr()
plt.figure(figsize=(15,5))
sns.heatmap(correlation, xticklabels = correlation.columns,
            yticklabels = correlation.columns,
            cmap = 'coolwarm', annot = True);
correlation

###  4.0.2 Genre Analysis

#### How are genres changing over time?

I've analyzed genre ties in this section. Because most of the movies in my datasets fit into more than one genre,it would be interesting to learn how the different types are related and perhaps check on how they affect the popularity of the movies compared against number votes.


In [ ]:
final_data.sort_values(by='release_date', ascending=True)

In [ ]:
genre_frequncy = dict(final_data['genres'].str.split(',', expand=True).stack().value_counts())
genre_frequncy

In [ ]:
fig, ax = plt.subplots(figsize =(15, 12))
explode = (0.0, 0.1, 0.2, 0.3, 0.2, 0.0,0.1,
           0.0, 0.2, 0.3, 0.0, 0.0,0.1, 0.0,
           0.2, 0.3, 0.5, 0.6,0.7, 0.8, 0.9, 1.0)

plt.pie(genre_frequncy.values(), labels=genre_frequncy.keys(),
        startangle=90, autopct='%.1f%%', colors=plt.cm.Set2.colors,
        explode = explode)

plt.show();

Drama movies had the highest frequency counts and it was the most distributed genre across the combinations.

#### Which genres have the highest correlation between budget and market success?

To find the most expensive movies based on production budget, I sort my production budget in descending order and checked for the top 100.

I am grouping my data by production budget to find out the top 100 expensive movies from my dataset then look into their genres to find out which genre appears most in the expensive movies.

In [ ]:
top_expensive_genre = final_data.sort_values(by=['production_budget'], ascending=False,).head(100)
print('These are the top 10 most expensive movies from my dataset')
top_expensive_genre.iloc[:,[6,1,2,3,4]].head(15)

In [ ]:
genres_in_top_expensive_movies = dict(top_expensive_genre['genres'].str.split(',', expand=True).stack().value_counts())

wordcloud = WordCloud(width = 800, height = 600,
                     collocations = False, 
                      ).generate_from_frequencies(genres_in_top_expensive_movies)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud);

This indicates, top perfoming genres are also the top expensive movies to produce.This means that in  majority of the cases, a larger budget has also resulted in a higher domestic box office gross.

Since most of the movies in the dataset have multiple Genres, I decided to split each of them to have a count plot and see how they are distributed.
From the value counts of the genres, drama had the highest value count while news had the least count.

#### average cost of producing movies

In [ ]:
production_cost = final_data.iloc[:,[6,1,2,3,4]]
print(f"The average cost of producing a movie is {round(production_cost.production_budget.mean())} in USD")

sns.set_style("whitegrid")
sns.boxplot(x = 'production_budget', data = production_cost);


In [ ]:
print(f"The average cost of producing a movie is {round(production_cost.production_budget.mean())} in USD")
print(f"The average domestic gross {round(production_cost.domestic_gross.mean())} in USD")
print(f"The average worldwide gross {round(production_cost.worldwide_gross.mean())} in USD")

plt.figure(figsize=(15,8))
ax = sns.boxplot(data=production_cost, color='orange', dodge=False);


### 4.0.3 Runtime Analysis.

In [ ]:
sns.boxplot(x=final_data["runtime_minutes"]);

In [ ]:
final_data['runtime_minutes'].mean()

The average runtime of most movies was at 100 minutes.I decided to check how it has changed over the years.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
sns.countplot(data=data['runtime_minutes'], x=data['year'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

Its is clear, the runtime for movies has not changed over the yaers, much.Its still 

### 4.0.4 Top movie stuido based on worlwide gross

In [ ]:
print(f"These are the top 15 movie studios from my dataset{pd.DataFrame(final_data.studio.value_counts()).head(15)}")

## 5.0 Conclusion

* Drama, Comedy and Action are the top genres. As seen in the pie chart and wordcloud
* Production budget and domestic gross have a high correlation.
* Drama genre continues to dominate most of the movies being produced.
* Runtime of movies has not greatly changed over the yaers as seen from the analysis.
* Significant number of movies go into loss.
* Movies that audience easily relates with are popular and have higher ratings.


## 6.0 Recommendations

* To enter into the film industry, it is important to consider short movies at an average of 50 minutes.

* There are factors that affect the movie perfomance after it has been prodced these include release date and the streaming platforms.

* More research could be done to understand what causes movies to be non profitable. This could help avoid loss in the movie industry.